# 基于方差分解的波动率溢出度量

First: 2024.4.10 Updated:2024.4.21 Version: 0.0.2 

```{contents}
```

## 介绍

- Sims(1980)引入了 VAR 方差分解，记录了某个变量 $i$ 的 $H$ 步前瞻预测误差方差（H-step-ahead forecast error variance ）中有多少是由另一个变量 $j$ 的创新造成的。
- Diebold & Yilmaz（2009）根据向量自回归（VAR）的预测误差方差分解，提出了一种波动溢出衡量方法。
- Diebold and Yilmaz (2012) 使用一个广义向量自回归框架，其中，预测误差方差分解对于变量排序是不变的，同时，提出了总体和方向波动溢出的度量。
- Gong et al.(2023) 研究涵盖了18个全球金融市场的实现波动率（RV）、隐含波动率（IV）和方差风险溢价（VRP）。文章使用了基于向量自回归（VAR）模型的方差分解框架中的溢出指数方法来计算不同金融市场之间的波动性溢出强度，并构建了多重网络模型来分析这些市场之间的连接性。研究发现，信息传输机制在RV、IV和VRP三个层面上存在差异。长期来看，IV层的溢出效应最强；短期来看，VRP层的溢出冲击最明显。在市场层面，欧洲和美国市场是主要的溢出发射者，而亚洲市场主要是溢出接收者。

## 向量自回归过程（VAR）

我们感兴趣建模一个 $T\times K$ 多元时间序列$Y$，其中 $T$ 表示观测样本数量，$K$ 表示变量数量。估计时间序列及其滞后值之间关系的一种方法是向量自回归过程：

$$
Y_t=\nu+A_1Y_{t-1}+\ldots+A_pY_{t-p}+u_t\\u_t\sim\mathsf{Normal}(0,\Sigma_u)
$$

其中 $A_i$ 是一个系数矩阵

VAR类假设传入的时间序列是平稳的。非平稳或趋势性数据通常可以通过首次差分或其他方法转换为平稳的。对于非平稳时间序列的直接分析，标准的稳定VAR(p)模型并不适用。

### 滞后阶数的选择

应用VAR模型，选择合适的滞后阶数（Lag order）p， 这里我们假设已经通过某种信息准则（如AIC）选择了最佳的滞后阶数

滞后阶数的选择可能是一个困难的问题。标准分析采用似然比检验或基于信息准则的阶数选择。

### 脉冲响应分析（IRA）
脉冲响应（Impulse Response）是对其中一个变量的单位脉冲的估计响应。在实践中，它们是使用$VAR(p)$过程的 $MA(\infty)$表示来计算的：

$$Y_t=\mu+\sum_{i=0}^\infty\Phi_iu_{t-i}$$

### 预测误差方法分解（FEVD）

使用正交化响应脉冲（Orthogonalized Impulse Response）${\Theta_i}$ 进行预测误差方法分解(Forecast Error Variance Decomposition, FEVD)

$$
\omega_{jk,i}=\sum_{i=0}^{h-1}(e_j^{\prime}\Theta_ie_k)^2/\mathrm{MSE}_j(h)
$$

$$\mathrm{MSE}_j(h)=\sum_{i=0}^{h-1}e_j^{\prime}\Phi_i\Sigma_u\Phi_i^{\prime}e_j$$

## 基于方差分解的波动率溢出度量

KPPS H-step-ahead forecast error variance decompositions:

$$\theta_{ij}^g(H)=\frac{\sigma_{jj}^{-1}\sum_{h=0}^{H-1}(e_i^{\prime}A_h\Sigma e_j)^2}{\sum_{h=0}^{H-1}(e_i^{\prime}A_h\Sigma A_h^{\prime}e_i)}$$

为了实现这个FEVD公式，我们需要先理解每个符号的含义和计算步骤：

- $ \theta_{ij}^g(H) $ 是广义预测误差方差分解（GFEVD）。
- $ \sigma_{jj}^{-1} $ 是残差协方差矩阵 $\Sigma$ 的逆矩阵中的对角元素。
- $ A_h $ 是VAR模型的冲击响应矩阵（Impulse Response Matrix）在第 $ h $ 步。
- $ e_i $ 和 $ e_j $ 是单位向量，用于选择对应的变量。

计算步骤：

1. 拟合VAR模型，得到模型参数。
2. 计算冲击响应矩阵 $ A_h $。
3. 根据公式计算每个变量的GFEVD。

